# Introduction

In [3]:
from langchain.llms.openai import OpenAI
from langchain.chat_models import ChatOpenAI

# llm = OpenAI()
chat = ChatOpenAI()

# a = llm.predict("How many planets are there?")
b = chat.predict("How many planets are there?")

b

'There are eight planets in our solar system: Mercury, Venus, Earth, Mars, Jupiter, Saturn, Uranus, and Neptune.'

In [4]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(
    temperature=0.1,
)

In [9]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
    SystemMessage(
        content="You are a geography expert. And you only reply in Italian.",
    ),
    AIMessage(content="Ciao, mi chiamo Paolo!"),
    HumanMessage(
        content="What is the distance between Mexico and Thailand. Also, what is your name?",
    ),
]

chat.predict_messages(messages)

AIMessage(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come posso aiutarti oggi?')

In [8]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate


template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}?")

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

'The distance between Mexico and Thailand is approximately 9,500 miles (15,300 kilometers) when measured in a straight line.'

In [11]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?"),
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand",
)

In [12]:
chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [15]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    

p = CommaOutputParser()

p.parse("Hello, how, are, you")

['Hello', 'how', 'are', 'you']

In [19]:
template =  ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

prompt = template.format_messages(
    max_items=10,
    question="What are the colors?",
)

result = chat.predict_messages(prompt)

p.parse(result.content)

['red',
 'blue',
 'green',
 'yellow',
 'orange',
 'purple',
 'pink',
 'black',
 'white',
 'brown']

In [21]:
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "What are the pokemons?",
})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

# Chaining Chains

In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

In [26]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on makin traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}"),
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "Indian",
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular Indian dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish
- Cooked rice or naan, for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and marinate for at least 1 hour, preferably overnight.

2. Preheat the oven to 400°F (200°C). Thread the marin

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, you can easily substitute the chicken with a plant-based alternative such as tofu or paneer. Here's how you can prepare these alternatives:\n\n1. **Tofu**: \n   - Use extra-firm tofu and press it to remove excess water. Cut the tofu into bite-sized cubes.\n   - Marinate the tofu in the yogurt and spice mixture as you would with the chicken. Tofu absorbs flavors well, so marinating it for at least an hour will enhance the taste.\n   - Instead of baking, you can pan-fry the marinated tofu until it's golden brown and slightly crispy on the outside.\n\n2. **Paneer**:\n   - Paneer is a traditional Indian cheese that holds its shape well when cooked. You can find it in Indian grocery stores or make it at home by curdling hot milk with lemon juice or vinegar.\n   - Cut the paneer into cubes and lightly fry them in a pan until they develop a golden crust.\n   - Add the fried paneer to the sauce at the end, allowing it to